# Random Forest Classification

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

## All CpG

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Fatigue
df = pd.read_csv('FatigueCateg_Methylome_APPRDXFilter.csv')
print(df.shape)
df.head(3)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Fatigue
(458, 31217)


,Sentrix,PATNO,HYS,Gender,APPRDX,MDSP_Fatigue,cg17960051,cg06562372,cg04099454,cg18542829,...,cg15500865,cg10329418,cg18241308,cg04350520,cg04685170,cg20985067,cg08461772,cg25879118,cg07510163,ch.13.693633F
0,200991620021_R01C01,3001,2,1,1,1,0.019716,0.024434,0.895146,0.024184,...,0.105174,0.262518,0.784256,0.886425,0.080717,0.814912,0.331552,0.916387,0.841837,0.036872
1,200973410167_R04C01,3007,2,1,1,1,0.020417,0.016055,0.882493,0.045616,...,0.066103,0.210723,0.905081,0.882225,0.082907,0.906400,0.497660,0.915460,0.822889,0.039866
2,200973410167_R05C01,3010,2,1,1,1,0.020654,0.017736,0.927835,0.047940,...,0.330337,0.310647,0.852778,0.876989,0.183659,0.917887,0.470535,0.894010,0.823575,0.036319


In [3]:
df1 = df.drop(['Sentrix','PATNO','HYS','Gender','MDSP_Fatigue'], axis=1)
df1.head(2)

,APPRDX,cg17960051,cg06562372,cg04099454,cg18542829,cg04599912,cg15759056,cg16703576,cg00031165,cg26835636,...,cg15500865,cg10329418,cg18241308,cg04350520,cg04685170,cg20985067,cg08461772,cg25879118,cg07510163,ch.13.693633F
0,1,0.019716,0.024434,0.895146,0.024184,0.014927,0.012150,0.026345,0.078477,0.016245,...,0.105174,0.262518,0.784256,0.886425,0.080717,0.814912,0.331552,0.916387,0.841837,0.036872
1,1,0.020417,0.016055,0.882493,0.045616,0.021537,0.013626,0.032826,0.033160,0.018156,...,0.066103,0.210723,0.905081,0.882225,0.082907,0.906400,0.497660,0.915460,0.822889,0.039866


In [4]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.475
2
Mean accuracy: 0.475
3
Mean accuracy: 0.465
4
Mean accuracy: 0.5650000000000002
5
Mean accuracy: 0.4950000000000001
6
Mean accuracy: 0.5599999999999999
7
Mean accuracy: 0.52
8
Mean accuracy: 0.475
9
Mean accuracy: 0.47000000000000003
10
Mean accuracy: 0.4999999999999999
11
Mean accuracy: 0.45
12
Mean accuracy: 0.45499999999999996
13
Mean accuracy: 0.5149999999999999
14
Mean accuracy: 0.48500000000000004
15
Mean accuracy: 0.5399999999999999
16
Mean accuracy: 0.5
17
Mean accuracy: 0.48
18
Mean accuracy: 0.525
19
Mean accuracy: 0.48999999999999994
20
Mean accuracy: 0.5800000000000001
21
Mean accuracy: 0.515
22
Mean accuracy: 0.5050000000000001
23
Mean accuracy: 0.5349999999999999
24
Mean accuracy: 0.49000000000000005
25
Mean accuracy: 0.545
26
Mean accuracy: 0.495
27
Mean accuracy: 0.555
28
Mean accuracy: 0.52
29
Mean accuracy: 0.5250000000000001
30
Mean accuracy: 0.47000000000000003
31
Mean accuracy: 0.5599999999999999
32
Mean accuracy: 0.45499999999999996
33
Mea

In [5]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Fatigue_AllCpG_100RF.csv", index = False)

0.5085000000000001


## RanFor 1000 times. Samples = 50

In [6]:
df2 = df1.drop (['APPRDX'], axis=1)

In [7]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(50, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.5900000000000001
2
Mean accuracy: 0.51
3
Mean accuracy: 0.55
4
Mean accuracy: 0.495
5
Mean accuracy: 0.515
6
Mean accuracy: 0.5249999999999999
7
Mean accuracy: 0.395
8
Mean accuracy: 0.515
9
Mean accuracy: 0.5249999999999999
10
Mean accuracy: 0.51
11
Mean accuracy: 0.43500000000000005
12
Mean accuracy: 0.53
13
Mean accuracy: 0.51
14
Mean accuracy: 0.545
15
Mean accuracy: 0.59
16
Mean accuracy: 0.55
17
Mean accuracy: 0.5400000000000001
18
Mean accuracy: 0.5399999999999999
19
Mean accuracy: 0.505
20
Mean accuracy: 0.54
21
Mean accuracy: 0.51
22
Mean accuracy: 0.54
23
Mean accuracy: 0.5299999999999999
24
Mean accuracy: 0.51
25
Mean accuracy: 0.5149999999999999
26
Mean accuracy: 0.495
27
Mean accuracy: 0.5349999999999999
28
Mean accuracy: 0.5700000000000001
29
Mean accuracy: 0.545
30
Mean accuracy: 0.545
31
Mean accuracy: 0.5449999999999999
32
Mean accuracy: 0.45
33
Mean accuracy: 0.505
34
Mean accuracy: 0.5000000000000001
35
Mean accuracy: 0.49499999999999994
36
Mean ac

In [8]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Fatigue_Rand50CpG_100RF.csv", index = False)

0.5093000000000001


## RanFor 100 times. Samples = 97

In [9]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(97, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.5549999999999999
2
Mean accuracy: 0.45499999999999996
3
Mean accuracy: 0.495
4
Mean accuracy: 0.4800000000000001
5
Mean accuracy: 0.51
6
Mean accuracy: 0.465
7
Mean accuracy: 0.39
8
Mean accuracy: 0.525
9
Mean accuracy: 0.5599999999999999
10
Mean accuracy: 0.5599999999999999
11
Mean accuracy: 0.45
12
Mean accuracy: 0.47000000000000003
13
Mean accuracy: 0.47000000000000003
14
Mean accuracy: 0.55
15
Mean accuracy: 0.5499999999999999
16
Mean accuracy: 0.565
17
Mean accuracy: 0.49000000000000005
18
Mean accuracy: 0.555
19
Mean accuracy: 0.45000000000000007
20
Mean accuracy: 0.505
21
Mean accuracy: 0.5549999999999999
22
Mean accuracy: 0.53
23
Mean accuracy: 0.52
24
Mean accuracy: 0.5
25
Mean accuracy: 0.43
26
Mean accuracy: 0.54
27
Mean accuracy: 0.5
28
Mean accuracy: 0.555
29
Mean accuracy: 0.55
30
Mean accuracy: 0.595
31
Mean accuracy: 0.5149999999999999
32
Mean accuracy: 0.525
33
Mean accuracy: 0.51
34
Mean accuracy: 0.52
35
Mean accuracy: 0.5449999999999999
36
Mean ac

In [10]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Fatigue_Rand97CpG_100RF.csv", index = False)

0.5064500000000001


## RanFor Top 50 Boruta 100 times. Samples = 50

In [11]:
df = pd.read_csv('Fatigue_Boruta_Top50_Data.csv')
print(df.shape)
df.head(2)

(458, 52)


,PATNO,APPRDX,cg06751221,cg03476938,cg23655939,cg21043558,cg19226007,cg18473117,cg12510028,cg27133496,...,cg15649411,cg03478448,cg09233704,cg22749855,cg05142211,cg00883689,cg12220605,cg12683454,cg16494828,cg15267345
0,3001,1,0.057391,0.549004,0.515701,0.667449,0.040020,0.938774,0.029810,0.510178,...,0.848940,0.636575,0.620103,0.843622,0.035018,0.525123,0.082447,0.023826,0.012698,0.029328
1,3007,1,0.083624,0.561954,0.523438,0.659961,0.017811,0.928175,0.035511,0.653650,...,0.888982,0.605065,0.682452,0.816596,0.041906,0.578082,0.113486,0.028970,0.012574,0.026110


In [12]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg06751221,cg03476938,cg23655939,cg21043558,cg19226007,cg18473117,cg12510028,cg27133496,cg17422176,...,cg15649411,cg03478448,cg09233704,cg22749855,cg05142211,cg00883689,cg12220605,cg12683454,cg16494828,cg15267345
0,1,0.057391,0.549004,0.515701,0.667449,0.040020,0.938774,0.029810,0.510178,0.117596,...,0.848940,0.636575,0.620103,0.843622,0.035018,0.525123,0.082447,0.023826,0.012698,0.029328
1,1,0.083624,0.561954,0.523438,0.659961,0.017811,0.928175,0.035511,0.653650,0.147173,...,0.888982,0.605065,0.682452,0.816596,0.041906,0.578082,0.113486,0.028970,0.012574,0.026110


In [13]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.67
2
Mean accuracy: 0.67
3
Mean accuracy: 0.645
4
Mean accuracy: 0.655
5
Mean accuracy: 0.6699999999999999
6
Mean accuracy: 0.6900000000000001
7
Mean accuracy: 0.6599999999999999
8
Mean accuracy: 0.6900000000000001
9
Mean accuracy: 0.6799999999999999
10
Mean accuracy: 0.655
11
Mean accuracy: 0.6399999999999999
12
Mean accuracy: 0.65
13
Mean accuracy: 0.6700000000000002
14
Mean accuracy: 0.62
15
Mean accuracy: 0.6449999999999999
16
Mean accuracy: 0.66
17
Mean accuracy: 0.64
18
Mean accuracy: 0.685
19
Mean accuracy: 0.66
20
Mean accuracy: 0.675
21
Mean accuracy: 0.635
22
Mean accuracy: 0.6050000000000001
23
Mean accuracy: 0.63
24
Mean accuracy: 0.6050000000000001
25
Mean accuracy: 0.655
26
Mean accuracy: 0.6450000000000001
27
Mean accuracy: 0.71
28
Mean accuracy: 0.6699999999999999
29
Mean accuracy: 0.6200000000000001
30
Mean accuracy: 0.635
31
Mean accuracy: 0.6799999999999999
32
Mean accuracy: 0.635
33
Mean accuracy: 0.6799999999999999
34
Mean accuracy: 0.64
35
Mean 

In [14]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Fatigue_Boruta50CpG_100RF.csv", index = False)

0.6532500000000003


## RanFor Top 50 NMI 100 times. Samples = 50

In [15]:
df = pd.read_csv('Fatigue_NMI_Top50_Data.csv')
print(df.shape)
df.head(2)

(458, 51)


,APPRDX,cg08178124,cg04721711,cg19460902,cg10044815,cg07940547,cg18473117,cg16448286,cg11224251,cg09377020,...,cg09652933,cg18838701,cg17458425,cg10783204,cg23290969,cg09826790,cg24224407,cg21049840,cg15942569,cg12526471
0,1,0.016319,0.831384,0.792809,0.894322,0.652888,0.938774,0.906014,0.661786,0.817618,...,0.014026,0.381468,0.856462,0.585492,0.035612,0.869270,0.875350,0.475823,0.888852,0.416863
1,1,0.023260,0.875973,0.621401,0.902732,0.615474,0.928175,0.890996,0.632961,0.800563,...,0.012004,0.289962,0.908599,0.654016,0.018923,0.914964,0.867527,0.405952,0.884358,0.449186


In [16]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.635
2
Mean accuracy: 0.67
3
Mean accuracy: 0.7050000000000001
4
Mean accuracy: 0.6900000000000001
5
Mean accuracy: 0.66
6
Mean accuracy: 0.7150000000000001
7
Mean accuracy: 0.7
8
Mean accuracy: 0.6950000000000001
9
Mean accuracy: 0.63
10
Mean accuracy: 0.6799999999999999
11
Mean accuracy: 0.705
12
Mean accuracy: 0.655
13
Mean accuracy: 0.7250000000000001
14
Mean accuracy: 0.67
15
Mean accuracy: 0.6699999999999999
16
Mean accuracy: 0.655
17
Mean accuracy: 0.6299999999999999
18
Mean accuracy: 0.6950000000000001
19
Mean accuracy: 0.715
20
Mean accuracy: 0.6849999999999999
21
Mean accuracy: 0.64
22
Mean accuracy: 0.645
23
Mean accuracy: 0.675
24
Mean accuracy: 0.6849999999999999
25
Mean accuracy: 0.705
26
Mean accuracy: 0.695
27
Mean accuracy: 0.7050000000000001
28
Mean accuracy: 0.6399999999999999
29
Mean accuracy: 0.6900000000000001
30
Mean accuracy: 0.665
31
Mean accuracy: 0.72
32
Mean accuracy: 0.655
33
Mean accuracy: 0.6799999999999999
34
Mean accuracy: 0.64
35
Mean

In [17]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Fatigue_NMI50CpG_100RF.csv", index = False)

0.6735499999999999


## RanFor Top 50 Boruta + NMI 100 times. Samples = 97

In [18]:
df = pd.read_csv('Fatigue_Top50sMerge_Data.csv')
print(df.shape)
df.head(2)

(458, 99)


,PATNO,APPRDX,cg06751221,cg03476938,cg23655939,cg21043558,cg19226007,cg18473117,cg12510028,cg27133496,...,cg09652933,cg18838701,cg17458425,cg10783204,cg23290969,cg09826790,cg24224407,cg21049840,cg15942569,cg12526471
0,3001,1,0.057391,0.549004,0.515701,0.667449,0.040020,0.938774,0.029810,0.510178,...,0.014026,0.381468,0.856462,0.585492,0.035612,0.869270,0.875350,0.475823,0.888852,0.416863
1,3007,1,0.083624,0.561954,0.523438,0.659961,0.017811,0.928175,0.035511,0.653650,...,0.012004,0.289962,0.908599,0.654016,0.018923,0.914964,0.867527,0.405952,0.884358,0.449186


In [19]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg06751221,cg03476938,cg23655939,cg21043558,cg19226007,cg18473117,cg12510028,cg27133496,cg17422176,...,cg09652933,cg18838701,cg17458425,cg10783204,cg23290969,cg09826790,cg24224407,cg21049840,cg15942569,cg12526471
0,1,0.057391,0.549004,0.515701,0.667449,0.040020,0.938774,0.029810,0.510178,0.117596,...,0.014026,0.381468,0.856462,0.585492,0.035612,0.869270,0.875350,0.475823,0.888852,0.416863
1,1,0.083624,0.561954,0.523438,0.659961,0.017811,0.928175,0.035511,0.653650,0.147173,...,0.012004,0.289962,0.908599,0.654016,0.018923,0.914964,0.867527,0.405952,0.884358,0.449186


In [20]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.6900000000000001
2
Mean accuracy: 0.675
3
Mean accuracy: 0.72
4
Mean accuracy: 0.6799999999999999
5
Mean accuracy: 0.67
6
Mean accuracy: 0.7150000000000001
7
Mean accuracy: 0.73
8
Mean accuracy: 0.6950000000000001
9
Mean accuracy: 0.65
10
Mean accuracy: 0.705
11
Mean accuracy: 0.685
12
Mean accuracy: 0.65
13
Mean accuracy: 0.735
14
Mean accuracy: 0.665
15
Mean accuracy: 0.7350000000000001
16
Mean accuracy: 0.6749999999999999
17
Mean accuracy: 0.665
18
Mean accuracy: 0.7099999999999999
19
Mean accuracy: 0.685
20
Mean accuracy: 0.725
21
Mean accuracy: 0.6399999999999999
22
Mean accuracy: 0.6799999999999999
23
Mean accuracy: 0.6849999999999999
24
Mean accuracy: 0.655
25
Mean accuracy: 0.755
26
Mean accuracy: 0.6599999999999999
27
Mean accuracy: 0.7050000000000001
28
Mean accuracy: 0.6799999999999999
29
Mean accuracy: 0.715
30
Mean accuracy: 0.6599999999999999
31
Mean accuracy: 0.7150000000000001
32
Mean accuracy: 0.655
33
Mean accuracy: 0.7150000000000001
34
Mean accura

In [21]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Fatigue_Top50sCpG97_100RF.csv", index = False)

0.6794999999999995
